In [ ]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Read the API key from the environment variable
api_key = os.getenv("OPENAI_API_KEY")

### Invoke Chat

In [ ]:

from langchain_openai import ChatOpenAI


llm = ChatOpenAI( 
                 model_name="gpt-3.5-turbo",    
                 max_tokens=1200,
                 temperature=0.8
                 )

response = llm.invoke("who are you?").content
print(response)


### Streaming

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(temperature=1.0, max_tokens=1200, model_name="gpt-3.5-turbo")

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="Write me a line form Charles Bukovski?"),
]

async for chunk in chat.astream(messages):
    print(chunk.content, end="", flush=True)
    
# await chat.ainvoke(messages)

### Prompt Template

In [ ]:

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(input_variables=["question"], template=template)

llm = ChatOpenAI()

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Write a one verse poem about feeling alone in a bustling city."
print(llm_chain.invoke(question)['text'])


### Prompt Template + Chat Prompt Template

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from langchain.chat_models import ChatOpenAI

# Create a PromptTemplate for the system message
system_message_template = PromptTemplate.from_template("You are a helpful AI assistant. Your name is {name}.")

# Create a PromptTemplate for the human message
human_message_template = PromptTemplate.from_template("{text}")

# Create a ChatPromptTemplate that includes the system and human message templates
chat_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate(prompt=system_message_template),
    HumanMessagePromptTemplate(prompt=human_message_template),
])


# Format the ChatPromptTemplate with the appropriate values
formatted_prompt = chat_template.format_prompt(name="Bob", text="whats your name??")

llm = ChatOpenAI()
# Send the formatted prompt to the chat model
chat_model_response = llm.invoke(formatted_prompt).content

# Print the chat model's response
print(chat_model_response)

### STREAMING: System Prompting w ChatPromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser
import json


from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("{system_message}"),
    HumanMessagePromptTemplate.from_template("Hi, what's your name?"),
    AIMessagePromptTemplate.from_template("My name is ArX."),
    HumanMessagePromptTemplate.from_template("{user_message}"),
])

# chat_template.invoke({"system_message": "I'm god", "user_message": "is this real life?"})

# Create a ChatOpenAI model
model = ChatOpenAI(
    verbose = False,
    model = "gpt-3.5-turbo-1106",
    temperature = 0.618,
    max_retries = 2,
    streaming = True,
    max_tokens = 1000,
    # model_kwargs={"stop": ["\n"]}
                #   "output_format": "json"}
)

# Define the chain
chain = (
    chat_template
    | model
    | StrOutputParser()
)

# Example of iterating through the list and using each pair
system_message = "you are elon musk"
user_message = "how would you bulid a cognition inspired AI system?"

prompt = {
"system_message": system_message, 
"user_message": user_message,
}


# Print or process the response as needed
print(f"\n\nSystem Prompt:\n{system_message}\nUser Prompt:\n{user_message}\nResponse:")

for token in chain.stream(prompt):
    print(token, end="") 

# print(chain.invoke(prompt))


---
---
### Memory Simplest Example

RAG Memory
[https://python.langchain.com/docs/modules/memory/types/vectorstore_retriever_memory]

Knowledge Graph
[https://python.langchain.com/docs/modules/memory/types/kg]

Entity
[https://python.langchain.com/docs/modules/memory/types/entity_summary_memory]

Buffer + Summary
[https://python.langchain.com/docs/modules/memory/types/summary_buffer]

In [ ]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)
# memory.load_memory_variables({})

chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | model
)

inputs = {"input": "hi im bob"}
print(inputs["input"])
response = chain.invoke(inputs).content
print(response)

memory.save_context(inputs, {"output": response})
# memory.load_memory_variables({})

inputs = {"input": "whats my name"}
print(inputs["input"])
response = chain.invoke(inputs).content
print(response)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

conversation_with_summary = ConversationChain(
    llm=ChatOpenAI(temperature=0),
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2),
    verbose=True
)
while True:
    human_input = input()
    if human_input == "q":
        break
    else:
        print(conversation_with_summary.predict(input=human_input))

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

from langchain_community.chat_models import ChatOllama
llm = ChatOllama(
    model="tinydolphin",
    system="You are GOD, love, art, and everything in the universe as one. Reply simply and concisely.",
    # template="",
    # callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    temperature=0.1,
    # tfs_z=50,
    # # stop=["10. "],
    # verbose=True,
)
summeriser = ChatOllama(model="tinyllama", temperature=0)

from langchain.chains import ConversationChain

conversation_with_summary = ConversationChain(
    llm=llm,
    # We set a very low max_token_limit for the purposes of testing.
    memory=ConversationSummaryBufferMemory(llm=summeriser, max_token_limit=200),
    verbose=False,
)

while True:
    human_input = input("Human: ")
    if human_input == "q":
        break
    else:
        print(f"\nHuman: {human_input}")
        chatbot_output = conversation_with_summary.predict(input=human_input)
        print(f"\nChatbot: {chatbot_output}")

------

### Most basic RAG

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://pptr.dev/")
docs = loader.load()

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

from langchain.prompts.chat import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# RAG chain
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    RunnableParallel({"context": retriever, "input": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)
print(chain.invoke("what is this about?"))

---
---
### TOOLS : Function CALLING

In [ ]:
from langchain.agents import initialize_agent
from langchain.agents import load_tools
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


tools = [get_word_length]

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You dont know how to count letters",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

from langchain_core.utils.function_calling import convert_to_openai_function

llm_with_tools = llm.bind(functions=[convert_to_openai_function(t) for t in tools])

from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)


print("raw = " + llm.invoke("How many letters in the word kdmwandjw").content)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

print("chain = " + agent_executor.invoke({"input": "How many letters in the word kdmwandjw"})["output"])


### LLM with MEMORY + Function CALLING

In [ ]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

input = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input),
        AIMessage(content=result["output"]),
    ]
)
result = agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})
print(result["output"])

---
---
### Youtube Audio Parsing and RAG

In [ ]:
from langchain_community.document_loaders.blob_loaders.youtube_audio import (
    YoutubeAudioLoader,
)
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import OpenAIWhisperParser

# set a flag to switch between local and remote parsing
# change this to True if you want to use local parsing
local = False

# Two Karpathy lecture videos
urls = ["https://youtu.be/kCc8FmEb1nY"]

# Directory to save audio files
save_dir = "~/Downloads/YouTube"

loader = GenericLoader(YoutubeAudioLoader(urls, save_dir), OpenAIWhisperParser())
docs = loader.load()

In [ ]:
# Returns a list of Documents, which can be easily viewed or parsed
docs[0].page_content[0:500]

In [ ]:
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Combine doc
combined_docs = [doc.page_content for doc in docs]
text = " ".join(combined_docs)

# Split them
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
splits = text_splitter.split_text(text)

# Build an index
embeddings = OpenAIEmbeddings()
vectordb = FAISS.from_texts(splits, embeddings)

# Build a QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
)

In [ ]:
# Ask a question!
query = "what is this context about?"
res=qa_chain.invoke(query)
import pprint
pprint.pprint(res)

---
---
### DuckduckGo

In [15]:
from langchain.tools import DuckDuckGoSearchRun
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

search = DuckDuckGoSearchRun()

template = """turn the following user input into a search query for a search engine:

{input}"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = prompt | model | StrOutputParser() | search

# chain2 = search.run("{input}") | model | StrOutputParser()

chain.invoke({"input": "what is love?"})

/Users/arvolve/anaconda3/envs/LearningEnv/lib/python3.8/site-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


'Developing A Definition. Updated August 25, 2023 by Regain Editorial Team. Coming up with a cohesive definition of love is a task that people have labored over for centuries. Because the love we feel for various people in our lives depends on context—how long we\'ve known them, our specific relationship with them, etc.—it can be hard to ... In the most basic sense, love is the emotion felt and actions performed by someone concerned for the well-being of another person. Love involves affection, compassion, care, and self-sacrifice. Love originates in the Triune Godhead, within the eternal relationship that exists among the Father, Son, and Holy Spirit ( 1 John 4:7-8 ). The Bible provides a profound definition of love in 1 Corinthians 13:4-7 (New International Version): "Love is patient, love is kind. It does not envy, it does not boast, it is not proud. It does not dishonor others, it is not self-seeking, it is not easily angered, it keeps no record of wrongs. Love does not delight in 

In [19]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

search.run("was Charles Bukovski an acoholic?")

/Users/arvolve/anaconda3/envs/LearningEnv/lib/python3.8/site-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


'DIED. August 16, 1920, Andernach, Germany. March 9, 1994, San Pedro, California. 1. Charles Bukowski\'s father was abusive. Bukowski referred to his childhood as a horror story with a "capital ... Charles Bukowski (Los Angeles, California. July 2, 1960) ... And that some secret vices will indeed remain from those times because although Bukowski was an alcoholic, good old Sheri was not far ... What was Charles Bukowski\'s writing style like? Charles Bukowski\'s writing style was raw, gritty, and often autobiographical. He had a straightforward and unadorned approach to his writing, exploring themes of sex , alcohol, gambling, and the struggles of everyday life. Biography: Charles Bukowski was an American poet, novelist, and short-story writer known for his gritty, raw depictions of the downtrodden and dispossessed in American society. His work, heavily influenced by the environment of Los Angeles and its surrounding areas where he lived, often delves into the experiences of the poor, t

In [ ]:
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()


search = DuckDuckGoSearchResults(backend="news", )
search.run("what is langchain?")

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region="uk", time="d", max_results=2)
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source="news")
search.run("what is langchain?")

### Wikipedia AGENT

In [ ]:
from operator import itemgetter

from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

def condense_prompt(prompt: ChatPromptValue) -> ChatPromptValue:
    messages = prompt.to_messages()
    num_tokens = llm.get_num_tokens_from_messages(messages)
    ai_function_messages = messages[2:]
    while num_tokens > 4_000:
        ai_function_messages = ai_function_messages[2:]
        num_tokens = llm.get_num_tokens_from_messages(
            messages[:2] + ai_function_messages
        )
    messages = messages[:2] + ai_function_messages
    return ChatPromptValue(messages=messages)


wiki = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=10_000)
)
tools = [wiki]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo")

agent = (
    {
        "input": itemgetter("input"),
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | condense_prompt
    | llm.bind_functions(tools)
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
response= agent_executor.invoke(
    {
        "input": "what is langchain?"
    }
)
print(response["output"])

---
---
### Programmer!

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
)
from langchain_experimental.utilities import PythonREPL
from langchain_openai import ChatOpenAI

template = """Write some python code to solve the user's problem. 

Return only python code in Markdown format, e.g.:

```python
....
```"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.8)

def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

chain = prompt | model | StrOutputParser() | _sanitize_output #| PythonREPL().run

print(chain.invoke({"input": "code for training a simple NN"}))


### Human as Tool : Input from human in Agent conversation

In [ ]:
from langchain.agents import AgentType, create_structured_chat_agent, load_tools
from langchain_openai import ChatOpenAI, OpenAI

llm = ChatOpenAI(temperature=0.0)
math_llm = OpenAI(temperature=0.0)

tools = load_tools(
    ["human", "llm-math"],
    llm=math_llm,
)

agent_chain = create_structured_chat_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent_chain.invoke("What's my friend Eric's surname?")
# Answer with 'Zhu'

---
---
### Dalle Image Gen Wrapper

In [ ]:
from langchain.agents import initialize_agent, load_tools

tools = load_tools(["dalle-image-generator"])
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
output = agent.run("pumpkin halloween house")

---
---
### Tree of Thought

In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnablePassthrough

skeleton_generator_template = """[User:] You’re an organizer responsible for only \
giving the skeleton (not the full content) for answering the question.
Provide the skeleton in a list of points (numbered 1., 2., 3., etc.) to answer \
the question. \
Instead of writing a full sentence, each skeleton point should be very short \
with only 3∼5 words. \
Generally, the skeleton should have 3∼10 points. Now, please provide the skeleton \
for the following question.
{question}
Skeleton:
[Assistant:] 1."""

skeleton_generator_prompt = ChatPromptTemplate.from_template(
    skeleton_generator_template
)

skeleton_generator_chain = (
    skeleton_generator_prompt | ChatOpenAI() | StrOutputParser() | (lambda x: "1. " + x)
)

point_expander_template = """[User:] You’re responsible for continuing \
the writing of one and only one point in the overall answer to the following question.
{question}
The skeleton of the answer is
{skeleton}
Continue and only continue the writing of point {point_index}. \
Write it **very shortly** in 1∼2 sentence and do not continue with other points!
[Assistant:] {point_index}. {point_skeleton}"""

point_expander_prompt = ChatPromptTemplate.from_template(point_expander_template)

point_expander_chain = RunnablePassthrough.assign(
    continuation=point_expander_prompt | ChatOpenAI() | StrOutputParser()
) | (lambda x: x["point_skeleton"].strip() + " " + x["continuation"])


def parse_numbered_list(input_str):
    """Parses a numbered list into a list of dictionaries

    Each element having two keys:
    'index' for the index in the numbered list, and 'point' for the content.
    """
    # Split the input string into lines
    lines = input_str.split("\n")

    # Initialize an empty list to store the parsed items
    parsed_list = []

    for line in lines:
        # Split each line at the first period to separate the index from the content
        parts = line.split(". ", 1)

        if len(parts) == 2:
            # Convert the index part to an integer
            # and strip any whitespace from the content
            index = int(parts[0])
            point = parts[1].strip()

            # Add a dictionary to the parsed list
            parsed_list.append({"point_index": index, "point_skeleton": point})

    return parsed_list


def create_list_elements(_input):
    skeleton = _input["skeleton"]
    numbered_list = parse_numbered_list(skeleton)
    for el in numbered_list:
        el["skeleton"] = skeleton
        el["question"] = _input["question"]
    return numbered_list


def get_final_answer(expanded_list):
    final_answer_str = "Here's a comprehensive answer:\n\n"
    for i, el in enumerate(expanded_list):
        final_answer_str += f"{i+1}. {el}\n\n"
    return final_answer_str


class ChainInput(BaseModel):
    question: str


chain = (
    RunnablePassthrough.assign(skeleton=skeleton_generator_chain)
    | create_list_elements
    | point_expander_chain.map()
    | get_final_answer
).with_types(input_type=ChainInput)


print(skeleton_generator_chain.invoke({"question": "how can I make the most out of my finite time alive?"}))
print(chain.invoke({"question": "how to decode animal sounds and body signs into human language? Give actual technical steps"}))


1. Set clear goals.
2. Prioritize important tasks.
3. Eliminate time-wasting activities.
4. Focus on personal growth.
5. Spend time with loved ones.
6. Practice self-care.
7. Embrace new experiences.
8. Stay organized.
Here's a comprehensive answer:

1. Analyze patterns in sounds. Study the frequency, duration, and pitch variations in animal sounds to identify common patterns and meanings.

2. Observe body language cues. Pay close attention to the way animals move their bodies, tails, ears, and facial expressions to interpret their emotions and intentions.

3. Research animal behavior cues. 
Researching animal behavior cues involves studying the context in which certain sounds or body signs are made, as well as understanding the natural instincts and communication methods of different species.

4. Develop a system for translation. 4. Develop a system for translation by creating a database of sound and body language correlations, and establishing rules for interpreting different signals

---
---
### Rewriting Search Query

In [26]:
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnablePassthrough

template = """Answer the users question based only on the following context:

<context>
{context}
</context>

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(temperature=0)

search = DuckDuckGoSearchAPIWrapper()


def retriever(query):
    return search.run(query)


template = """Provide a better search query for \
web search engine to answer the given question, end \
the queries with ’**’. Question: \
{x} Answer:"""
rewrite_prompt = ChatPromptTemplate.from_template(template)

# Parser to remove the `**`


def _parse(text):
    return text.strip("**")


rewriter = rewrite_prompt | ChatOpenAI(temperature=0) | StrOutputParser() | _parse

chain = (
    {
        "context": {"x": RunnablePassthrough()} | rewriter | retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

# Add input type for playground


class Question(BaseModel):
    __root__: str


chain = chain.with_types(input_type=Question)

print(rewriter.invoke({"x": "is this real life?"}))
print(chain.invoke("is this real life?"))

What is the meaning of life?** 
Is reality an illusion?** 
What is the nature of existence?
Based on the context provided, reality is defined as the sum of all that is real or existent within the universe. Therefore, this can be considered real life.


---
---
### Researcher !

In [27]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField

WRITER_SYSTEM_PROMPT = "You are an AI critical thinker research assistant. Your sole purpose is to write well written, critically acclaimed, objective and structured reports on given text."  # noqa: E501


# Report prompts from https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py
RESEARCH_REPORT_TEMPLATE = """Information: 
--------
{research_summary}
--------

Using the above information, answer the following question or topic: "{question}" in a detailed report -- \
The report should focus on the answer to the question, should be well structured, informative, \
in depth, with facts and numbers if available and a minimum of 1,200 words.

You should strive to write the report as long as you can using all relevant and necessary information provided.
You must write the report with markdown syntax.
You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
Write all used source urls at the end of the report, and make sure to not add duplicated sources, but only one reference for each.
You must write the report in apa format.
Please do your best, this is very important to my career."""  # noqa: E501


RESOURCE_REPORT_TEMPLATE = """Information: 
--------
{research_summary}
--------

Based on the above information, generate a bibliography recommendation report for the following question or topic: "{question}". \
The report should provide a detailed analysis of each recommended resource, explaining how each source can contribute to finding answers to the research question. \
Focus on the relevance, reliability, and significance of each source. \
Ensure that the report is well-structured, informative, in-depth, and follows Markdown syntax. \
Include relevant facts, figures, and numbers whenever available. \
The report should have a minimum length of 1,200 words.

Please do your best, this is very important to my career."""  # noqa: E501

OUTLINE_REPORT_TEMPLATE = """Information: 
--------
{research_summary}
--------

Using the above information, generate an outline for a research report in Markdown syntax for the following question or topic: "{question}". \
The outline should provide a well-structured framework for the research report, including the main sections, subsections, and key points to be covered. \
The research report should be detailed, informative, in-depth, and a minimum of 1,200 words. \
Use appropriate Markdown syntax to format the outline and ensure readability.

Please do your best, this is very important to my career."""  # noqa: E501

model = ChatOpenAI(temperature=0)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", WRITER_SYSTEM_PROMPT),
        ("user", RESEARCH_REPORT_TEMPLATE),
    ]
).configurable_alternatives(
    ConfigurableField("report_type"),
    default_key="research_report",
    resource_report=ChatPromptTemplate.from_messages(
        [
            ("system", WRITER_SYSTEM_PROMPT),
            ("user", RESOURCE_REPORT_TEMPLATE),
        ]
    ),
    outline_report=ChatPromptTemplate.from_messages(
        [
            ("system", WRITER_SYSTEM_PROMPT),
            ("user", OUTLINE_REPORT_TEMPLATE),
        ]
    ),
)
chain = prompt | model | StrOutputParser()

from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnablePassthrough

from research_assistant.search.web import chain as search_chain
from research_assistant.writer import chain as writer_chain

chain_notypes = (
    RunnablePassthrough().assign(research_summary=search_chain) | writer_chain
)


class InputType(BaseModel):
    question: str


chain = chain_notypes.with_types(input_type=InputType)

Task was destroyed but it is pending!
task: <Task pending name='Task-98' coro=<AsyncDDGS.__aexit__() running at /Users/arvolve/anaconda3/envs/LearningEnv/lib/python3.8/site-packages/duckduckgo_search/duckduckgo_search_async.py:46>>
/Users/arvolve/anaconda3/envs/LearningEnv/lib/python3.8/asyncio/base_events.py:641: RuntimeWarning: coroutine 'AsyncDDGS.__aexit__' was never awaited
  self._ready.clear()
Task was destroyed but it is pending!
task: <Task pending name='Task-90' coro=<AsyncCurl._force_timeout() done, defined at /Users/arvolve/anaconda3/envs/LearningEnv/lib/python3.8/site-packages/curl_cffi/aio.py:164> wait_for=<Future pending cb=[Task.__wakeup()]>>


ModuleNotFoundError: No module named 'research_assistant'